In [1]:
import numpy as np
from numpy import load
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from prl_utils import (
    Mode,
    get_features,
    get_labels,
    normalize_train_labels,
    normalize_val_labels,
)

In [22]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense, Dropout
import numpy as np

class BlockwiseGRU(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, num_blocks):
        """
        GRU model that processes time series data block-by-block.

        Args:
        - input_dim (int): Feature dimension per time step.
        - hidden_dim (int): Hidden size of GRU.
        - output_dim (int): Number of output variables.
        - num_blocks (int): Number of time series blocks.
        """
        super(BlockwiseGRU, self).__init__()
        
        self.num_blocks = num_blocks
        self.hidden_dim = hidden_dim
        
        # Shared GRU layer (processes one block at a time)
        self.gru = GRU(hidden_dim, return_sequences=True, return_state=True)
        self.gru_dropout = Dropout(0.5)
        self.dense1 = Dense(int(hidden_dim/2), activation='relu')
        self.dropout1 = Dropout(0.5)
        self.dense2 = Dense(int(hidden_dim/4), activation='relu')
        self.dropout2 = Dropout(0.5)
        # Final output layer (fully connected)
        self.output_layer = Dense(output_dim)

    def call(self, x):
        """
        Forward pass.

        Args:
        - x (Tensor): Shape (batch_size, num_blocks, time_steps_per_block, input_dim)

        Returns:
        - output (Tensor): Shape (batch_size, output_dim)
        """
        print(x.shape)
        batch_size, num_blocks, time_steps, _ = x.shape

        # Initialize hidden state (default to zeros)
        hidden_state = tf.zeros((batch_size, self.hidden_dim))
        b_outputs = []

        for i in range(num_blocks):
            block = x[:, i, :, :]  # Extract the i-th block: (batch_size, time_steps, input_dim)
            _, hidden_state = self.gru(block, initial_state=hidden_state)  # Update hidden state
            b_outputs.append(hidden_state)
        
        # Concatenate the outputs of the GRU layers or global average pooling
        rnn_outputs = tf.keras.layers.Concatenate(axis=1)(b_outputs)
        print(rnn_outputs.shape)
        rnn_outputs = self.gru_dropout(rnn_outputs)
        # Dense layers
        d_outputs_2 = self.dense1(rnn_outputs)
        d_outputs_2 = self.dropout1(d_outputs_2)
        d_outputs_2 = self.dense2(d_outputs_2)
        d_outputs_2 = self.dropout2(d_outputs_2)        
        # Use the final hidden state to make a prediction
        output = self.output_layer(d_outputs_2)
        
        return output





# Example Usage
batch_size = 25
num_blocks = 3
time_steps_per_block = 20
input_dim = 16
hidden_dim = 10
output_dim = 3

# Create model
best_model = BlockwiseGRU(input_dim, hidden_dim, output_dim, num_blocks)

# Generate dummy y labels (batch_size, output_dim)
y = np.random.randn(100, output_dim).astype(np.float32)

# Generate dummy input data (100 agents, num_blocks, time_steps_per_block, input_dim)
x = np.random.randn(100, num_blocks, time_steps_per_block, input_dim).astype(np.float32)


# Compile Model with Custom Loss
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='mse')

# Train the model
history = best_model.fit(
    x=x, 
    y=y, 
    epochs=10, 
    batch_size=batch_size)



# # Forward pass
# output = model(x)
# print(output.shape)  # Expected: (batch_size, output_dim)

Epoch 1/10


2025-02-07 14:48:55.442558: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:48:55.443624: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:48:55.444383: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(25, 3, 20, 16)


2025-02-07 14:48:55.526313: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:48:55.527320: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:48:55.527903: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(25, 30)
(25, 3, 20, 16)


2025-02-07 14:48:55.999894: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:48:56.000895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:48:56.001427: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(25, 30)
4/4 [==============================] - 2s 12ms/step - loss: 1.1762
Epoch 2/10
4/4 [==============================] - 0s 6ms/step - loss: 1.1331
Epoch 3/10
4/4 [==============================] - 0s 6ms/step - loss: 1.2727
Epoch 4/10
4/4 [==============================] - 0s 6ms/step - loss: 1.4099
Epoch 5/10
4/4 [==============================] - 0s 6ms/step - loss: 1.4228
Epoch 6/10
4/4 [==============================] - 0s 6ms/step - loss: 1.2146
Epoch 7/10
4/4 [==============================] - 0s 6ms/step - loss: 1.4835
Epoch 8/10
4/4 [==============================] - 0s 7ms/step - loss: 1.3860
Epoch 9/10
4/4 [==============================] - 0s 6ms/step - loss: 1.2050
Epoch 10/10
4/4 [==============================] - 0s 6ms/step - loss: 1.6057


In [23]:
print(tf.__version__)

2.12.0


In [25]:
# Create model
best_model = BlockwiseGRU(input_dim, hidden_dim, output_dim, num_blocks)
# Generate dummy y labels (batch_size, output_dim)
y = np.random.randn(100, output_dim).astype(np.float32)

# Generate dummy input data (batch_size, num_blocks, time_steps_per_block, input_dim)
x = np.random.randn(100, num_blocks, time_steps_per_block, input_dim).astype(np.float32)


# Compile Model with Custom Loss
best_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
              loss='mse')
history = best_model.fit(
    x=x, 
    y=y, 
    epochs=10, 
    batch_size=2)

Epoch 1/10


2025-02-07 14:51:23.858805: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:51:23.859591: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:51:23.860186: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(2, 3, 20, 16)


2025-02-07 14:51:23.939663: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:51:23.940506: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:51:23.941125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(2, 30)
(2, 3, 20, 16)


2025-02-07 14:51:24.381438: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2025-02-07 14:51:24.382675: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2025-02-07 14:51:24.383365: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

(2, 30)
50/50 [==============================] - 2s 6ms/step - loss: 1.8918
Epoch 2/10
50/50 [==============================] - 0s 5ms/step - loss: 1.2362
Epoch 3/10
50/50 [==============================] - 0s 6ms/step - loss: 1.3833
Epoch 4/10
50/50 [==============================] - 0s 5ms/step - loss: 1.1919
Epoch 5/10
50/50 [==============================] - 0s 5ms/step - loss: 1.1068
Epoch 6/10
50/50 [==============================] - 0s 5ms/step - loss: 1.0439
Epoch 7/10
50/50 [==============================] - 0s 5ms/step - loss: 1.2645
Epoch 8/10
50/50 [==============================] - 0s 5ms/step - loss: 1.0087
Epoch 9/10
50/50 [==============================] - 0s 5ms/step - loss: 1.1570
Epoch 10/10
50/50 [==============================] - 0s 5ms/step - loss: 1.2665


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import GRU, Dense
import numpy as np

class ConditionAwareGRU(tf.keras.Model):
    def __init__(self, input_dim, hidden_dim, output_dim, num_blocks):
        """
        GRU model with condition-based masking.
        
        Args:
        - input_dim (int): Feature dimension per time step.
        - hidden_dim (int): Hidden size of GRU.
        - output_dim (int): Number of inferred variables (A, B, C).
        - num_blocks (int): Number of time series blocks.
        """
        super(ConditionAwareGRU, self).__init__()
        
        self.num_blocks = num_blocks
        self.hidden_dim = hidden_dim
        
        # Shared GRU layer (processes one block at a time)
        self.gru = GRU(hidden_dim, return_sequences=True, return_state=True)
        
        # Separate output layers for condition-dependent and condition-agnostic variables
        self.output_A = Dense(1)  # Variable A (Condition 1)
        self.output_B = Dense(1)  # Variable B (Condition 2)
        self.output_C = Dense(1)  # Variable C (Condition agnostic)

    def call(self, x, conditions):
        """
        Forward pass.
        
        Args:
        - x (Tensor): Shape (num_agents, num_blocks, time_steps_per_block, input_dim)
        - conditions (Tensor): Shape (num_agents, num_blocks), binary condition indicator (1 = Condition 1, 0 = Condition 2)

        Returns:
        - output (Tensor): Shape (num_agents, 3) for (A, B, C)
        """
        num_agents, num_blocks, time_steps, _ = x.shape

        # Initialize hidden states
        hidden_state = tf.zeros((num_agents, self.hidden_dim))

        # Store hidden states for condition-based processing
        hidden_states_A = []  # For Variable A (Condition 1)
        hidden_states_B = []  # For Variable B (Condition 2)
        hidden_states_C = []  # For Variable C (Condition agnostic)

        # Process each block sequentially
        for i in range(num_blocks):
            block = x[:, i, :, :]  # Extract i-th block (num_agents, time_steps, input_dim)
            _, hidden_state = self.gru(block, initial_state=hidden_state)  # Update hidden state
            
            # Extract the condition mask
            condition_mask = tf.expand_dims(conditions[:, i], axis=-1)  # Shape (num_agents, 1)

            # Store condition-masked hidden states
            hidden_states_A.append(hidden_state * condition_mask)      # Only update A when Condition 1
            hidden_states_B.append(hidden_state * (1 - condition_mask)) # Only update B when Condition 2
            hidden_states_C.append(hidden_state)                        # Always update C

        # Aggregate hidden states
        final_A = tf.reduce_sum(tf.stack(hidden_states_A, axis=1), axis=1)  # Sum over Condition 1 blocks
        final_B = tf.reduce_sum(tf.stack(hidden_states_B, axis=1), axis=1)  # Sum over Condition 2 blocks
        final_C = tf.reduce_mean(tf.stack(hidden_states_C, axis=1), axis=1) # Mean over all blocks

        # Predict variables
        var_A = self.output_A(final_A)  # Variable A
        var_B = self.output_B(final_B)  # Variable B
        var_C = self.output_C(final_C)  # Variable C

        # Concatenate outputs into (num_agents, 3)
        output = tf.concat([var_A, var_B, var_C], axis=-1)
        
        return output

# Example Usage
num_agents = 3
num_blocks = 3
time_steps_per_block = 20
input_dim = 16
hidden_dim = 32
output_dim = 3  # A, B, C

# Create model
model = ConditionAwareGRU(input_dim, hidden_dim, output_dim, num_blocks)

# Generate dummy input data (num_agents, num_blocks, time_steps_per_block, input_dim)
x = np.random.randn(num_agents, num_blocks, time_steps_per_block, input_dim).astype(np.float32)

# Generate random conditions (1 = Condition 1, 0 = Condition 2) for each block per agent
conditions = np.random.randint(0, 2, size=(num_agents, num_blocks)).astype(np.float32)

# Forward pass
output = model(x, conditions)
print(output.shape)  # Expected: (num_agents, 3)

(3, 3)
